In [2]:
import spacy
from spacy.lang.sv.stop_words import STOP_WORDS
from collections import Counter
import pandas as pd

from swinno_bioeconomy_directionality.swinno_helpers import connect_swinno_db

conn = connect_swinno_db()


In [7]:
bvs = pd.read_sql(
    """
SELECT
  innovation.sinno_id,
  use_sectors.use_sectors,
  innovation.[Innovation name in Swedish] as name,
  innovation.[Description in Swedish] as desc,
  innovation.[Additional information if Origin = New scientific discovery] || innovation.[Additional information if Origin = New technologies or materials] || innovation.[Additional info if Origin = Official regulation, legislation and standards.] || innovation.[Additional information if Origin = Solution for a problem.] || innovation.[Additional information if Origin = Performance] || innovation.[Additional information if Origin = Other] AS info
from
  innovation
  JOIN use_sectors on use_sectors.sinno_id = innovation.sinno_id
WHERE
  innovation.sinno_id IN (
    SELECT
      sinno_id
    from
      bioeconomy_visions_articles
  );""",
    conn,
)


In [9]:
bvs.head()


,sinno_id,use_sectors,name,desc,info
0,6000001,21219,Ensocoat/Ansogloss/Tambrite,Ny typ av kartongpapper i tre varianter avsedd...,
1,6039001,28,WEN,Pneumatisk graverpenna. Drivenheten har inga a...,
2,6341001,20,SawInfo,System för lagerhantering av virkespaket. Med ...,
3,6562001,21,Diodair,Gasmätare för mätning av gaser med hjälp av la...,Ny kombination av tekniker.
4,6740001,01,Frosty,Universalterrängfordon på band. Maskinen har t...,


In [10]:
nlp = spacy.load("sv_core_news_sm")


In [28]:
def get_keywords(text):
    doc = nlp(text.lower())
    tokens = [
        token.text
        for token in doc
        if not token.is_stop and not token.is_punct and token.is_alpha
    ]
    word_freq = Counter(tokens)
    common_words = word_freq.most_common(5)
    return common_words, tokens


In [72]:
doc = nlp(bvs.desc[0].lower())


In [79]:
tokens = [token.text for token in doc]
tokens


['ny',
 'typ',
 'av',
 'kartongpapper',
 'i',
 'tre',
 'varianter',
 'avsedd',
 'för',
 'förpackingsindustrin',
 '.',
 'kartongpapperet',
 'skapar',
 'nya',
 'möjligheter',
 'för',
 'digitaltryck',
 'för',
 'kartong',
 'genom',
 'att',
 'den',
 'är',
 'specialanpassad',
 'för',
 'digitaltryck',
 'i',
 'xerox',
 'igen3',
 '.']

In [80]:
no_stop = [token.text for token in doc if not token.is_stop and not token.is_punct]
no_stop


['ny',
 'typ',
 'kartongpapper',
 'varianter',
 'avsedd',
 'förpackingsindustrin',
 'kartongpapperet',
 'skapar',
 'nya',
 'möjligheter',
 'digitaltryck',
 'kartong',
 'är',
 'specialanpassad',
 'digitaltryck',
 'xerox',
 'igen3']

In [76]:
no_punct_tokens = [token.text for token in doc if not token.is_punct]
no_punct_tokens


['ny',
 'typ',
 'av',
 'kartongpapper',
 'i',
 'tre',
 'varianter',
 'avsedd',
 'för',
 'förpackingsindustrin',
 'kartongpapperet',
 'skapar',
 'nya',
 'möjligheter',
 'för',
 'digitaltryck',
 'för',
 'kartong',
 'genom',
 'att',
 'den',
 'är',
 'specialanpassad',
 'för',
 'digitaltryck',
 'i',
 'xerox',
 'igen3']

['ny',
 'typ',
 'kartongpapper',
 'varianter',
 'avsedd',
 'förpackingsindustrin',
 'kartongpapperet',
 'skapar',
 'nya',
 'möjligheter',
 'digitaltryck',
 'kartong',
 'är',
 'specialanpassad',
 'digitaltryck',
 'xerox']

In [30]:
print(t)


['ny', 'typ', 'kartongpapper', 'varianter', 'avsedd', 'förpackingsindustrin', 'kartongpapperet', 'skapar', 'nya', 'möjligheter', 'digitaltryck', 'kartong', 'är', 'specialanpassad', 'digitaltryck', 'xerox']


In [31]:
print(c)


[('digitaltryck', 2), ('ny', 1), ('typ', 1), ('kartongpapper', 1), ('varianter', 1)]


In [116]:
def get_hotwords(text):
    pos_tags = ["PROPN", "ADJ", "NOUN"]  # 1
    doc = nlp(text.lower())  # 2
    result = [
        token.lemma_
        for token in doc
        if not token.is_stop
        and not token.is_punct
        and token.is_alpha
        and token.pos_ in pos_tags
    ]

    return result  # 5


In [117]:
bvs["key_words"] = bvs["desc"].apply(get_hotwords)


In [118]:
bvs.head()


,sinno_id,use_sectors,name,desc,info,key_words
0,6000001,21219,Ensocoat/Ansogloss/Tambrite,Ny typ av kartongpapper i tre varianter avsedd...,,"[ny, typ, kartongpapp, variant, avse, förpacki..."
1,6039001,28,WEN,Pneumatisk graverpenna. Drivenheten har inga a...,,"[pneumatisk, graverpenna, drivenhet, rörlig, d..."
2,6341001,20,SawInfo,System för lagerhantering av virkespaket. Med ...,,"[system, lagerhantering, virkespak, hjälp, sys..."
3,6562001,21,Diodair,Gasmätare för mätning av gaser med hjälp av la...,Ny kombination av tekniker.,"[gasmätare, mätning, gas, hjälp, laserdiod, fr..."
4,6740001,01,Frosty,Universalterrängfordon på band. Maskinen har t...,,"[universalterrängfordon, band, maskin, bred, l..."


In [128]:
# join all words in key words list into one string across rows
key_words = [word for row in bvs["key_words"] for word in row]
# copy a list to clipboard


In [131]:
key_word_counts = Counter(key_words).most_common(100)

# dict from list of tuples
key_word_counts = dict(key_word_counts)
key_words_df = pd.DataFrame.from_dict(key_word_counts, orient="index").reset_index()
key_words_df.head()


,index,0
0,ny,360
1,maskin,279
2,metod,198
3,träd,198
4,system,135


In [ ]:
key_words_df.loc[:, "index"].to_clipboard(index=False, header=False)


In [122]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
sns.barplot(
    x=list(key_word_counts.values()), y=list(key_word_counts.keys()), orient="h"
)


<AxesSubplot: >

In [98]:
bvs.iloc[0]["desc"]


'Ny typ av kartongpapper i tre varianter avsedd för förpackingsindustrin. Kartongpapperet skapar nya möjligheter för digitaltryck för kartong genom att den är specialanpassad för digitaltryck i Xerox Igen3.'

In [105]:
all_kw = dict(Counter(key_words))


In [108]:
sum_kw = sum(all_kw.values())

kw_freq = {k: v / sum_kw for k, v in all_kw.items()}


In [115]:
key_word_doc = nlp(" ".join(key_words))
[token.lemma_ for token in key_word_doc]


['ny',
 'typ',
 'kartongpappa',
 'variant',
 'avse',
 'förpackingsindustrin',
 'kartongpapper',
 'ny',
 'möjlighet',
 'digitaltryck',
 'kartong',
 'specialanpassa',
 'digitaltryck',
 'xerox',
 'pneumatisk',
 'graverpenna',
 'drivenhet',
 'rörlig',
 'del',
 'tallrik',
 'stiftet',
 'fästad',
 'återföringsfjäd',
 'gravyr',
 'verktygsstål',
 'system',
 'lagerhantering',
 'virkespak',
 'hjälp',
 'system',
 'sågverk',
 'ordning',
 'strölagda',
 'virkespak',
 'problem',
 'sågverk',
 'gasmät',
 'mätning',
 'gas',
 'hjälp',
 'laserdioda',
 'främst',
 'fördel',
 'gas',
 'laserdiod',
 'kort',
 'reaktionstide',
 'metod',
 'processindustri',
 'svar',
 'laserdiodteknik',
 'process',
 'svår',
 'omöjlig',
 'metod',
 'industrimiljö',
 'universalterrängfordon',
 'band',
 'maskin',
 'bred',
 'larvband',
 'individuell',
 'hydrostatisk',
 'drift',
 'hastighetsreglering',
 'steglös',
 'lasermätningssystem',
 'bearbetning',
 'propelleraxla',
 'mätenhet',
 'stålprofila',
 'tak',
 'vägg',
 'värmningssystem',
 

## Using Chat GPT


In [8]:
bvs.iloc[:5]["desc"].to_clipboard(index=False, header=False, encoding="utf-8")
